In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/massp-housing-prices-in-melbourne/submission.csv
/kaggle/input/massp-housing-prices-in-melbourne/train.csv
/kaggle/input/massp-housing-prices-in-melbourne/test.csv


In [2]:
import os
import warnings
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from IPython.display import display
from pandas.api.types import CategoricalDtype

from category_encoders import MEstimateEncoder
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.feature_selection import mutual_info_regression
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor

# Set Matplotlib defaults
plt.style.use("seaborn-whitegrid")
plt.rc("figure", autolayout=True)
plt.rc(
    "axes",
    labelweight="bold",
    labelsize="large",
    titleweight="bold",
    titlesize=14,
    titlepad=10,
)

# Mute warnings
warnings.filterwarnings('ignore')

# Tổng quan dữ liệu

In [3]:
data_dir = Path("../input/massp-housing-prices-in-melbourne/")
df_train = pd.read_csv(data_dir / "train.csv", index_col="id")
df_test = pd.read_csv(data_dir / "test.csv", index_col="id")
df = pd.concat((df_train, df_test))

In [4]:
def missing_values_table(df):
    # Total missing values
    mis_val = df.isnull().sum()       
    # Percentage of missing values
    mis_val_percent = 100 * df.isnull().sum() / len(df)       
    # Make a table with the results
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)        
    # Rename the columns
    mis_val_table_ren_columns = mis_val_table.rename(
    columns = {0 : 'Missing Values', 1 : '% of Total Values'})        
    # Sort the table by percentage of missing descending
    mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
    '% of Total Values', ascending=False).round(1)
    return mis_val_table_ren_columns

In [5]:
df

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
id,,,,,,,,,,,,,,,,,,,,,
0,Pascoe Vale,1/22 Archibald St,3,t,900000.0,S,Nelson,12/08/2017,8.5,3044.0,...,2.0,1.0,0.0,149.0,2014.0,Moreland City Council,-37.72882,144.93301,Northern Metropolitan,7485.0
1,Northcote,3 Lorna Av,2,h,1800000.0,S,McGrath,18/03/2017,5.5,3070.0,...,NaN,NaN,NaN,NaN,NaN,Darebin City Council,NaN,NaN,Northern Metropolitan,11364.0
2,Aspendale,75 Laura St,3,h,988000.0,S,Biggin,23/09/2017,21.5,3195.0,...,NaN,NaN,NaN,NaN,NaN,Kingston City Council,NaN,NaN,South-Eastern Metropolitan,2824.0
3,Ormond,2 Stewart St,3,h,940000.0,PI,Buxton,16/07/2016,11.8,3204.0,...,1.0,1.0,397.0,124.0,1940.0,Glen Eira City Council,-37.90640,145.02910,Southern Metropolitan,3578.0
4,Coburg,171 The Avenue,3,h,1035000.0,S,Nelson,24/02/2018,6.7,3058.0,...,1.0,2.0,375.0,169.0,1930.0,Darebin City Council,-37.75339,144.97687,Northern Metropolitan,11204.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27242,Ashwood,95 Huntingdale Rd,4,h,NaN,VB,Tim,10/12/2016,12.2,3147.0,...,2.0,2.0,803.0,152.0,1970.0,Monash City Council,-37.86210,145.11260,Southern Metropolitan,2894.0
27243,Hawthorn,23 Edgerton St,4,h,NaN,S,Kay,17/09/2016,4.6,3122.0,...,2.0,1.0,218.0,173.0,1890.0,Boroondara City Council,-37.81420,145.02520,Southern Metropolitan,11308.0
27244,Windsor,106/233 Dandenong Rd,1,u,NaN,VB,Ray,15/10/2016,5.1,3181.0,...,1.0,1.0,0.0,NaN,2010.0,Port Phillip City Council,-37.85840,144.99940,Southern Metropolitan,4380.0


In [6]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Rooms,27247.0,2.992293e+00,0.954795,1.00000,2.0000,3.00000,4.000000e+00,1.600000e+01
Price,13623.0,1.047997e+06,635527.348323,85000.00000,637000.0000,870000.00000,1.285000e+06,8.000000e+06
Distance,27246.0,1.128028e+01,6.787469,0.00000,6.4000,10.50000,1.400000e+01,4.810000e+01
Postcode,27246.0,3.113796e+03,111.139676,3000.00000,3046.0000,3088.00000,3.153000e+03,3.978000e+03
Bedroom2,20806.0,3.046237e+00,0.955122,0.00000,2.0000,3.00000,4.000000e+00,2.000000e+01
Bathroom,20800.0,1.591683e+00,0.700880,0.00000,1.0000,1.00000,2.000000e+00,9.000000e+00
Car,20423.0,1.715370e+00,0.994216,0.00000,1.0000,2.00000,2.000000e+00,1.800000e+01
Landsize,17982.0,5.934889e+02,3757.266429,0.00000,219.0000,512.00000,6.640000e+02,4.330140e+05
BuildingArea,10656.0,1.568346e+02,449.222791,0.00000,100.0000,133.00000,1.820000e+02,4.451500e+04
YearBuilt,12084.0,1.966609e+03,36.762373,1196.00000,1950.0000,1970.00000,2.000000e+03,2.019000e+03


In [7]:
missing_values_table(df)

,Missing Values,% of Total Values
BuildingArea,16591,60.9
YearBuilt,15163,55.7
Price,13624,50.0
Landsize,9265,34.0
Car,6824,25.0
Bathroom,6447,23.7
Bedroom2,6441,23.6
Lattitude,6254,23.0
Longtitude,6254,23.0
CouncilArea,3,0.0


# Tiền xử lý dữ liệu

In [8]:
def load_data():
    # Read data
    data_dir = Path("../input/massp-housing-prices-in-melbourne/")
    df_train = pd.read_csv(data_dir / "train.csv", index_col="id")
    df_test = pd.read_csv(data_dir / "test.csv", index_col="id")
    # Merge the splits so we can process them together
    df = pd.concat((df_train, df_test))
    # Preprocessing
    df = clean(df)
    df = impute(df)
    df = encode(df)
    # Reform splits
    df_train = df.loc[df_train.index, :]
    df_test = df.loc[df_test.index, :]
    return df_train, df_test

### Clean data

In [9]:
def clean(df):
    # Một số hàng có giá trị trường "BuildingArea" là 0 -> sửa về median
    df.loc[df.BuildingArea == 0]['BuildingArea'] = df.BuildingArea.median()
    return df

### Xử lý dữ liệu thiếu

In [10]:
fill_median_nan = ['BuildingArea', 'Landsize', 'YearBuilt']
fill_min_nan = ['Car', 'Bedroom2', 'Bathroom', 'Rooms']
fill_freq_nan = ['CouncilArea', 'Regionname', 'Propertycount', 'Distance', 'Postcode', 'Lattitude', 'Longtitude']

In [11]:
def impute(df):
    for name in fill_median_nan:
        df[name] = df[name].fillna(df[name].median())
        df[name+"_NAN"] = df[name].isnull()*1
    for name in fill_min_nan:
        df[name] = df[name].fillna(df[name].min())
        df[name+"_NAN"] = df[name].isnull()*1
    for name in fill_freq_nan:
        df[name] = df[name].fillna(df[name].mode()[0])
        df[name+"_NAN"] = df[name].isnull()*1
    return df

### Định dạng lại một số trường

In [12]:
categorical_features = df.select_dtypes(exclude=[np.number])
categorical_features.columns

Index(['Suburb', 'Address', 'Type', 'Method', 'SellerG', 'Date', 'CouncilArea',
       'Regionname'],
      dtype='object')

In [13]:
categoricals = ['Suburb', 'Type', 'Method', 'SellerG', 'CouncilArea',
       'Regionname', 'Postcode']

def encode(df):
    # Change categoricals to type 'category'
    for name in categoricals:
        df[name] = df[name].astype("category")
        # Add a None category for missing values
        if "None" not in df[name].cat.categories:
            df[name].cat.add_categories("None", inplace=True)
    # Change 'Date' type to datetime
    df['Date'] = pd.to_datetime(df['Date'])
    df['Month'] = df['Date'].dt.month
    df['Year'] = df['Date'].dt.year
    df.drop('Date', axis='columns', inplace=True)
    return df

## Load data

In [14]:
df_train, df_test = load_data()
df_train

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Distance,Postcode,Bedroom2,...,Rooms_NAN,CouncilArea_NAN,Regionname_NAN,Propertycount_NAN,Distance_NAN,Postcode_NAN,Lattitude_NAN,Longtitude_NAN,Month,Year
id,,,,,,,,,,,,,,,,,,,,,
0,Pascoe Vale,1/22 Archibald St,3,t,900000.0,S,Nelson,8.5,3044.0,3.0,...,0,0,0,0,0,0,0,0,12,2017
1,Northcote,3 Lorna Av,2,h,1800000.0,S,McGrath,5.5,3070.0,0.0,...,0,0,0,0,0,0,0,0,3,2017
2,Aspendale,75 Laura St,3,h,988000.0,S,Biggin,21.5,3195.0,0.0,...,0,0,0,0,0,0,0,0,9,2017
3,Ormond,2 Stewart St,3,h,940000.0,PI,Buxton,11.8,3204.0,3.0,...,0,0,0,0,0,0,0,0,7,2016
4,Coburg,171 The Avenue,3,h,1035000.0,S,Nelson,6.7,3058.0,3.0,...,0,0,0,0,0,0,0,0,2,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13618,Fitzroy,22/26 Victoria St,2,u,660000.0,SP,Jellis,2.1,3065.0,0.0,...,0,0,0,0,0,0,0,0,10,2017
13619,Coburg North,17 Mehegan Av,3,h,765000.0,S,Raine,9.2,3058.0,0.0,...,0,0,0,0,0,0,0,0,8,2016
13620,Hawthorn East,29 Fairmount Rd,3,h,3600000.0,S,Marshall,6.2,3123.0,3.0,...,0,0,0,0,0,0,0,0,9,2017


In [15]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13624 entries, 13623 to 27246
Data columns (total 36 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   Suburb             13624 non-null  category
 1   Address            13624 non-null  object  
 2   Rooms              13624 non-null  int64   
 3   Type               13624 non-null  category
 4   Price              0 non-null      float64 
 5   Method             13624 non-null  category
 6   SellerG            13624 non-null  category
 7   Distance           13624 non-null  float64 
 8   Postcode           13624 non-null  category
 9   Bedroom2           13624 non-null  float64 
 10  Bathroom           13624 non-null  float64 
 11  Car                13624 non-null  float64 
 12  Landsize           13624 non-null  float64 
 13  BuildingArea       13624 non-null  float64 
 14  YearBuilt          13624 non-null  float64 
 15  CouncilArea        13624 non-null  category
 16  

In [16]:
df_train.describe().T

,count,mean,std,min,25%,50%,75%,max
Rooms,13623.0,2.989870e+00,0.960175,1.00000,2.00000,3.0000,4.000000e+00,1.600000e+01
Price,13623.0,1.047997e+06,635527.348323,85000.00000,637000.00000,870000.0000,1.285000e+06,8.000000e+06
Distance,13623.0,1.121588e+01,6.731874,0.00000,6.40000,10.4000,1.400000e+01,4.810000e+01
Bedroom2,13623.0,2.327901e+00,1.534181,0.00000,1.00000,3.0000,3.000000e+00,1.600000e+01
Bathroom,13623.0,1.219408e+00,0.912128,0.00000,1.00000,1.0000,2.000000e+00,8.000000e+00
Car,13623.0,1.293548e+00,1.142957,0.00000,0.00000,1.0000,2.000000e+00,1.000000e+01
Landsize,13623.0,5.772569e+02,4031.296426,0.00000,348.00000,512.0000,5.915000e+02,4.330140e+05
BuildingArea,13623.0,1.397454e+02,77.946326,0.00000,133.00000,133.0000,1.330000e+02,6.791000e+03
YearBuilt,13623.0,1.968719e+03,23.846098,1820.00000,1970.00000,1970.0000,1.970000e+03,2.019000e+03
Lattitude,13623.0,-3.781358e+01,0.080050,-38.18415,-37.84235,-37.8361,-3.776584e+01,-3.740720e+01


In [17]:
missing_values_table(df_train)

,Missing Values,% of Total Values


## Thiết lập Baseline

In [18]:
def score_dataset(X, y, model=XGBRegressor()):
    for colname in X.select_dtypes(["category"]):
        X[colname] = X[colname].cat.codes
    # Metric for Housing competition is RMSLE (Root Mean Squared Log Error)
    log_y = np.log(y)
    score = cross_val_score(
        model, X, log_y, cv=5, scoring="neg_mean_squared_error",
    )
    score = -1 * score.mean()
    score = np.sqrt(score)
    return score

In [19]:
X = df_train.copy()
y = X.pop('Price')
X.columns.values

array(['Suburb', 'Address', 'Rooms', 'Type', 'Method', 'SellerG',
       'Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Car', 'Landsize',
       'BuildingArea', 'YearBuilt', 'CouncilArea', 'Lattitude',
       'Longtitude', 'Regionname', 'Propertycount', 'BuildingArea_NAN',
       'Landsize_NAN', 'YearBuilt_NAN', 'Car_NAN', 'Bedroom2_NAN',
       'Bathroom_NAN', 'Rooms_NAN', 'CouncilArea_NAN', 'Regionname_NAN',
       'Propertycount_NAN', 'Distance_NAN', 'Postcode_NAN',
       'Lattitude_NAN', 'Longtitude_NAN', 'Month', 'Year'], dtype=object)

In [20]:
X_test = X.drop('Address', axis='columns')

In [21]:
baseline_score = score_dataset(X_test, y)
print(f"Baseline score: {baseline_score:.5f} RMSLE")

Baseline score: 0.20120 RMSLE


# Feature Engineering

In [22]:
def encode_address(c):
    try:
        if 'Avenue' in c or 'Av' in c:
            return 'Av'
    except TypeError:
        print(c)
    if 'St' in c:
        return 'St'
    if 'Rd' in c:
        return 'Rd'
    if 'Pde' in c:
        return 'Pde'
    if 'Pl' in c:
        return 'Pl'
    if 'Esplanade' in c or 'Esp' in c:
        return 'Esp'
    s = c.split()[-1]
    if s == 'Outlook':
        return 'Out'
    if s == 'Loop':
        return 'Lp'
    if s == 'Nook':
        return 'Nk'
    if s == 'Circuit':
        return 'Cct'
    if s == 'Terrace':
        return 'Tce'
    if s in ['Grove', 'Gve']:
        return 'Gr'
    if s == 'Greenway':
        return 'Gwy'
    if s in['Avenue', 'Aveue', 'Ave']:
        return 'Av'
    if s == 'Boulevard':
        return 'Bvd'
    if s in ['Street', 'street']:
        return 'St'
    if s == 'Crescent': # Crossway
        return 'Cr'
    if s == 'Wy':
        return 'Wky'
    if s == 'Highway':
        return 'Hwy'
    if s in ['Ridge', 'Ri', 'Ridgeway']:
        return 'Rdg'
    if s == 'Esplanade':
        return 'Esp'
    if s == 'Mews':
        return 'Mw'
    if s == 'Grange':
        return 'Gra'
    if s == 'Views':
        return 'Vw'
    if s == 'Parkway':
        return 'Pky'
    if s == 'Glade':
        return 'Gld'
    if 'The' in c:
        return 'The'
    return s

def to_season(month):
    if month in range(1, 4):
        return 'Spring'
    if month in range(4, 7):
        return 'Summer'
    if month in range(7, 10):
        return 'Fall'
    if month in range(10, 13):
        return 'Winter'

In [23]:
df_train, df_test = load_data()
n_train = df_train.shape[0]
n_test = df_test.shape[0]
df = pd.concat([df_train, df_test])

df['Address'] = np.vectorize(encode_address)(df['Address'])
df['Address'] = df['Address'].astype('category')
df['Season'] = np.vectorize(to_season)(df.Month)
df['Season'] = df['Season'].astype("category")
df['Spacious'] = df.Landsize + df.BuildingArea
df['Age'] = np.where(2019 - df.YearBuilt > 50, "Historic", "Contemporary")
df['Age'] = df['Age'].astype("category")
df_train = df[:n_train]
df_test = df[n_train:]

In [24]:
df_test.shape

(13624, 39)

In [25]:
X = df_train.copy()
y = X.pop('Price')
X.columns.values

array(['Suburb', 'Address', 'Rooms', 'Type', 'Method', 'SellerG',
       'Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Car', 'Landsize',
       'BuildingArea', 'YearBuilt', 'CouncilArea', 'Lattitude',
       'Longtitude', 'Regionname', 'Propertycount', 'BuildingArea_NAN',
       'Landsize_NAN', 'YearBuilt_NAN', 'Car_NAN', 'Bedroom2_NAN',
       'Bathroom_NAN', 'Rooms_NAN', 'CouncilArea_NAN', 'Regionname_NAN',
       'Propertycount_NAN', 'Distance_NAN', 'Postcode_NAN',
       'Lattitude_NAN', 'Longtitude_NAN', 'Month', 'Year', 'Season',
       'Spacious', 'Age'], dtype=object)

In [26]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13623 entries, 0 to 13622
Data columns (total 38 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   Suburb             13623 non-null  category
 1   Address            13623 non-null  category
 2   Rooms              13623 non-null  int64   
 3   Type               13623 non-null  category
 4   Method             13623 non-null  category
 5   SellerG            13623 non-null  category
 6   Distance           13623 non-null  float64 
 7   Postcode           13623 non-null  category
 8   Bedroom2           13623 non-null  float64 
 9   Bathroom           13623 non-null  float64 
 10  Car                13623 non-null  float64 
 11  Landsize           13623 non-null  float64 
 12  BuildingArea       13623 non-null  float64 
 13  YearBuilt          13623 non-null  float64 
 14  CouncilArea        13623 non-null  category
 15  Lattitude          13623 non-null  float64 
 16  Long

In [27]:
features = ['Suburb', 'Address', 'Rooms', 'Type', 'Method', 'SellerG',
       'Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Car', 'Landsize',
       'BuildingArea', 'YearBuilt', 'CouncilArea', 'Lattitude',
       'Longtitude', 'Regionname', 'Propertycount', 'BuildingArea_NAN',
       'Landsize_NAN', 'YearBuilt_NAN', 'Car_NAN', 'Bedroom2_NAN',
       'Bathroom_NAN', 'Rooms_NAN', 'CouncilArea_NAN', 'Regionname_NAN',
       'Propertycount_NAN', 'Distance_NAN', 'Postcode_NAN',
       'Lattitude_NAN', 'Longtitude_NAN', 'Month', 'Year', 'Season', 'Age']

X.loc[:, features]
        
score1 = score_dataset(X.loc[:, features], y)
print(f"Score: {score1:.5f} RMSLE")

Score: 0.20093 RMSLE


In [28]:
import optuna

def objective(trial):
    xgb_params = dict(
        max_depth=trial.suggest_int("max_depth", 2, 10),
        learning_rate=trial.suggest_float("learning_rate", 1e-4, 1e-1, log=True),
        n_estimators=trial.suggest_int("n_estimators", 1000, 8000),
        min_child_weight=trial.suggest_int("min_child_weight", 1, 10),
        colsample_bytree=trial.suggest_float("colsample_bytree", 0.2, 1.0),
        subsample=trial.suggest_float("subsample", 0.2, 1.0),
        reg_alpha=trial.suggest_float("reg_alpha", 1e-4, 1e2, log=True),
        reg_lambda=trial.suggest_float("reg_lambda", 1e-4, 1e2, log=True),
    )
    xgb = XGBRegressor(**xgb_params)
    return score_dataset(X.loc[:, features], y, xgb)

# study = optuna.create_study(direction="minimize")
# study.optimize(objective, n_trials=20)
# xgb_params = study.best_params

In [29]:
xgb_params = {'max_depth': 9,
 'learning_rate': 0.020416492248752128,
 'n_estimators': 6543,
 'min_child_weight': 9,
 'colsample_bytree': 0.4100894611473811,
 'subsample': 0.9104292269474314,
 'reg_alpha': 3.2401418058918092,
 'reg_lambda': 0.1066050759436948}

In [30]:
X_train, X_test = df_train.loc[:,features], df_test.loc[:,features]
for colname in X_train.select_dtypes(["category"]):
    X_train[colname] = X_train[colname].cat.codes
for colname in X_test.select_dtypes(["category"]):
    X_test[colname] = X_test[colname].cat.codes

In [31]:
xgb = XGBRegressor(**xgb_params)
xgb.fit(X_train, np.log(y))
predictions = np.exp(xgb.predict(X_test))

output = pd.DataFrame({'id': X_test.index, 'Price': predictions})
output.to_csv('my_submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!


In [32]:
output.shape

(13624, 2)